In [1]:
import time
import torch.backends.cudnn as cudnn
from torch import nn
from easydict import EasyDict as edict
from models import Generator, Discriminator, TruncatedVGG19
from datasets import SRDataset
from utils import *
from solver import train, train_pre

/home/shaw/anaconda3/envs/hw5/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# config
config = edict()
config.csv_folder = './SRDataset'
config.HR_data_folder = './SRDataset/div2k/DIV2K_train_HR'
config.LR_data_folder = './SRDataset/div2k/DIV2K_train_LR_bicubic/X4'
config.crop_size = 96
config.scaling_factor = 4

# Generator parameters
config.G = edict()
config.G.large_kernel_size = 9
config.G.small_kernel_size = 3
config.G.n_channels = 64
config.G.n_blocks = 16

# Discriminator parameters
config.D = edict()
config.D.kernel_size = 3
config.D.n_channels = 64
config.D.n_blocks = 8
config.D.fc_size = 1024

# Learning parameters
config.checkpoint = None # path to model (SRGAN) checkpoint, None if none
config.batch_size = 16 #16
config.start_epoch = 0
config.epochs = 200 #200
config.workers = 16  # 4
config.vgg19_i = 5  # the index i in the definition for VGG loss; see paper
config.vgg19_j = 4  # the index j in the definition for VGG loss; see paper
config.beta = 1e-3  # the coefficient to weight the adversarial loss in the perceptual loss
config.print_freq =100 #50
config.lr = 1e-3

# Default device
config.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
cudnn.benchmark = True

In [3]:
if config.checkpoint is None:
    # Generator
    generator = Generator(config)

    # optimizer for pre-train
    optimizer_p= torch.optim.Adam(params=filter(lambda p: p.requires_grad, generator.parameters()),
                                   lr=config.lr)

    # Initialize generator's optimizer
    optimizer_g = torch.optim.Adam(params=filter(lambda p: p.requires_grad, generator.parameters()),
                                   lr=config.lr)

    # Discriminator
    discriminator = Discriminator(config)
    optimizer_d = torch.optim.Adam(params=filter(lambda p: p.requires_grad, discriminator.parameters()),
                                   lr=config.lr)

else:
    checkpoint = torch.load(config.checkpoint)
    config.start_epoch = checkpoint['epoch'] + 1
    generator = checkpoint['generator']
    discriminator = checkpoint['discriminator']
    optimizer_p = checkpoint['optimizer_p']
    optimizer_g = checkpoint['optimizer_g']
    optimizer_d = checkpoint['optimizer_d']
    print("\nLoaded checkpoint from epoch %d.\n" % (checkpoint['epoch'] + 1))

In [4]:
# Truncated VGG19 network to be used in the loss calculation
truncated_vgg19 = TruncatedVGG19(i=config.vgg19_i, j=config.vgg19_j)
truncated_vgg19.eval()

/home/shaw/anaconda3/envs/hw5/lib/python3.9/site-packages/torchvision/models/_utils.py:135: UserWarning: Using 'weights' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(
/home/shaw/anaconda3/envs/hw5/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


35


TruncatedVGG19(
  (vgg): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1)

In [5]:
# Loss functions
content_loss_criterion = nn.MSELoss()
pre_train_loss_criterion = nn.MSELoss()
adversarial_loss_criterion = nn.BCEWithLogitsLoss()

In [6]:
# Move to default device
generator = generator.to(config.device)
discriminator = discriminator.to(config.device)
truncated_vgg19 = truncated_vgg19.to(config.device)
content_loss_criterion = content_loss_criterion.to(config.device)
adversarial_loss_criterion = adversarial_loss_criterion.to(config.device)

In [7]:
# Custom dataloaders
train_dataset = SRDataset(split='train', config=config)
train_loader = torch.utils.data.DataLoader(train_dataset,
                                           batch_size=config.batch_size,
                                           shuffle=True, 
                                           num_workers=config.workers,
                                           pin_memory=True)

In [8]:
# Epochs
for epoch in range(config.start_epoch, config.epochs):
    # At the halfway point, reduce learning rate to a tenth
    if epoch == int(config.epochs / 4 + 1):
        adjust_learning_rate(optimizer_p, 0.1)
    if epoch == int(config.epochs / 2 + 1):
        adjust_learning_rate(optimizer_g, 0.1)
        adjust_learning_rate(optimizer_d, 0.1)
    # One epoch's training
    train_pre(train_loader=train_loader,
          generator=generator,
          discriminator=discriminator,
          truncated_vgg19=truncated_vgg19,
          pre_train_loss_criterion = pre_train_loss_criterion,
          content_loss_criterion=content_loss_criterion,
          adversarial_loss_criterion=adversarial_loss_criterion,
          optimizer_p=optimizer_p,
          optimizer_g=optimizer_g,
          optimizer_d=optimizer_d,
          epoch=epoch,
          device=config.device,
          beta=config.beta,
          print_freq=config.print_freq,
          max_epoch = config.epochs)
    # Save checkpoint
    torch.save({'epoch': epoch,
                'generator': generator,
                'discriminator': discriminator,
                'optimizer_p': optimizer_p,
                'optimizer_g': optimizer_g,
                'optimizer_d': optimizer_d},
                'checkpoint_srgan.pth.tar')
    # Save checkpoint every 10 epoch
    if (epoch+1)%10 == 0 :
        torch.save({'epoch': epoch,
                    'generator': generator,
                    'discriminator': discriminator,
                    'optimizer_p': optimizer_p,
                    'optimizer_g': optimizer_g,
                    'optimizer_d': optimizer_d},
                    'checkpoint_srgan.{}.pth.tar'.format((epoch+1)))

Epoch: [0][0/5000]----Batch Time 5.078 (5.078)----Data Time 2.603 (2.603)----Cont. Loss 0.5733 (0.5733)
Epoch: [0][100/5000]----Batch Time 0.092 (0.138)----Data Time 0.012 (0.033)----Cont. Loss 0.4469 (0.4288)
Epoch: [0][200/5000]----Batch Time 0.071 (0.127)----Data Time 0.000 (0.031)----Cont. Loss 0.4155 (0.4169)
Epoch: [0][300/5000]----Batch Time 0.113 (0.125)----Data Time 0.002 (0.033)----Cont. Loss 0.4662 (0.4086)
Epoch: [0][400/5000]----Batch Time 0.084 (0.121)----Data Time 0.000 (0.031)----Cont. Loss 0.3227 (0.4083)
Epoch: [0][500/5000]----Batch Time 0.111 (0.121)----Data Time 0.000 (0.032)----Cont. Loss 0.4031 (0.4075)
Epoch: [0][600/5000]----Batch Time 0.068 (0.120)----Data Time 0.000 (0.034)----Cont. Loss 0.4564 (0.4113)
Epoch: [0][700/5000]----Batch Time 0.350 (0.120)----Data Time 0.274 (0.035)----Cont. Loss 0.4091 (0.4090)
Epoch: [0][800/5000]----Batch Time 0.074 (0.119)----Data Time 0.000 (0.034)----Cont. Loss 0.2435 (0.4072)
Epoch: [0][900/5000]----Batch Time 0.113 (0.118)

OutOfMemoryError: CUDA out of memory. Tried to allocate 1.14 GiB (GPU 0; 7.80 GiB total capacity; 710.94 MiB already allocated; 1.15 GiB free; 770.00 MiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF